<a href="https://colab.research.google.com/github/praymesh/VLM-SEGMENTATION/blob/main/Visual_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
LM